In [1]:
import numpy as np
import pandas as pd
import re
import datetime


In [2]:
data = pd.read_csv("data/approval_polllist.csv")
data.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Joe Biden,All polls,3/23/2022,1/19/2021,1/21/2021,Morning Consult,B,15000.0,a,0.259361,...,28.0,49.069948,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74272,139491,1/28/2021,16:23:18 23 Mar 2022
1,Joe Biden,All polls,3/23/2022,1/19/2021,1/21/2021,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.338188,...,45.0,50.040379,39.216619,NaN,T,https://www.rasmussenreports.com/public_conten...,74247,139395,1/22/2021,16:23:18 23 Mar 2022
2,Joe Biden,All polls,3/23/2022,1/20/2021,1/21/2021,YouGov,B+,1516.0,a,1.245401,...,28.0,46.263649,28.659628,NaN,NaN,https://docs.cdn.yougov.com/u3h9dresbn/2021012...,74327,139570,2/2/2021,16:23:18 23 Mar 2022
3,Joe Biden,All polls,3/23/2022,1/20/2021,1/22/2021,Morning Consult,B,15000.0,a,0.233261,...,28.0,50.069948,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74273,139492,1/28/2021,16:23:18 23 Mar 2022
4,Joe Biden,All polls,3/23/2022,1/20/2021,1/21/2021,Morning Consult,B,1993.0,rv,0.092978,...,31.0,55.069948,34.211739,NaN,NaN,https://assets.morningconsult.com/wp-uploads/2...,74246,139394,1/22/2021,16:23:18 23 Mar 2022


In [3]:
data['startdate'].nunique()

427

In [4]:
data['startdate'].unique()

array(['1/19/2021', '1/20/2021', '1/21/2021', '1/22/2021', '1/23/2021',
       '1/24/2021', '1/25/2021', '1/26/2021', '1/27/2021', '1/28/2021',
       '1/29/2021', '1/30/2021', '1/31/2021', '2/1/2021', '2/2/2021',
       '2/3/2021', '2/4/2021', '2/5/2021', '2/6/2021', '2/7/2021',
       '2/8/2021', '2/9/2021', '2/10/2021', '2/11/2021', '2/12/2021',
       '2/13/2021', '2/14/2021', '2/15/2021', '2/16/2021', '2/17/2021',
       '2/18/2021', '2/19/2021', '2/20/2021', '2/21/2021', '2/22/2021',
       '2/23/2021', '2/24/2021', '2/25/2021', '2/26/2021', '2/27/2021',
       '2/28/2021', '3/1/2021', '3/2/2021', '3/3/2021', '3/4/2021',
       '3/6/2021', '3/5/2021', '3/7/2021', '3/8/2021', '3/9/2021',
       '3/10/2021', '3/11/2021', '3/12/2021', '3/13/2021', '3/14/2021',
       '3/15/2021', '3/16/2021', '3/17/2021', '3/18/2021', '3/19/2021',
       '3/21/2021', '3/20/2021', '3/22/2021', '3/23/2021', '3/24/2021',
       '3/25/2021', '3/26/2021', '3/28/2021', '3/27/2021', '3/29/2021',
       '3/

In [5]:
data.shape

(3115, 22)

## 2022 removed

In [6]:
data = data[data['enddate'].str.contains('2021')]
data.shape

(2529, 22)

In [7]:
# create month column
data['month'] = data['enddate']
data['month'].nunique()
# data.head()

345

In [8]:
# data['month'] = pd.to_datetime(data['enddate']).dt.to_period('M')
data['month'] = pd.DatetimeIndex(data['enddate']).month

data.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp,month
0,Joe Biden,All polls,3/23/2022,1/19/2021,1/21/2021,Morning Consult,B,15000.0,a,0.259361,...,49.069948,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74272,139491,1/28/2021,16:23:18 23 Mar 2022,1
1,Joe Biden,All polls,3/23/2022,1/19/2021,1/21/2021,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.338188,...,50.040379,39.216619,NaN,T,https://www.rasmussenreports.com/public_conten...,74247,139395,1/22/2021,16:23:18 23 Mar 2022,1
2,Joe Biden,All polls,3/23/2022,1/20/2021,1/21/2021,YouGov,B+,1516.0,a,1.245401,...,46.263649,28.659628,NaN,NaN,https://docs.cdn.yougov.com/u3h9dresbn/2021012...,74327,139570,2/2/2021,16:23:18 23 Mar 2022,1
3,Joe Biden,All polls,3/23/2022,1/20/2021,1/22/2021,Morning Consult,B,15000.0,a,0.233261,...,50.069948,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74273,139492,1/28/2021,16:23:18 23 Mar 2022,1
4,Joe Biden,All polls,3/23/2022,1/20/2021,1/21/2021,Morning Consult,B,1993.0,rv,0.092978,...,55.069948,34.211739,NaN,NaN,https://assets.morningconsult.com/wp-uploads/2...,74246,139394,1/22/2021,16:23:18 23 Mar 2022,1


In [9]:
data['month'].astype(str)

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       2
26       1
27       1
28       1
29       1
        ..
2935    12
2936    12
2937    12
2938    12
2939    12
2940    12
2941    12
2942    12
2943    12
2944    12
2945    12
2946    12
2947    12
2948    12
2949    12
2950    12
2951    12
2952    12
2953    12
2954    12
2955    12
2956    12
2957    12
2958    12
2959    12
2960    12
2961    12
2962    12
2963    12
2964    12
Name: month, Length: 2529, dtype: object

In [10]:
# def convert_num_to_month_name():
mapping = {
    '1': 'January',
    '2': 'February', 
    '3': 'March',
    '4': 'April',
    '5': 'May',
    '6': 'June',
    '7': 'July',
    '8': 'August',
    '9': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December'
}
  

In [11]:
data.dtypes

president               object
subgroup                object
modeldate               object
startdate               object
enddate                 object
pollster                object
grade                   object
samplesize             float64
population              object
weight                 float64
influence              float64
approve                float64
disapprove             float64
adjusted_approve       float64
adjusted_disapprove    float64
multiversions           object
tracking                object
url                     object
poll_id                  int64
question_id              int64
createddate             object
timestamp               object
month                    int64
dtype: object

In [12]:
data['enddate'] = pd.to_datetime(data['enddate'])
data.dtypes

president                      object
subgroup                       object
modeldate                      object
startdate                      object
enddate                datetime64[ns]
pollster                       object
grade                          object
samplesize                    float64
population                     object
weight                        float64
influence                     float64
approve                       float64
disapprove                    float64
adjusted_approve              float64
adjusted_disapprove           float64
multiversions                  object
tracking                       object
url                            object
poll_id                         int64
question_id                     int64
createddate                    object
timestamp                      object
month                           int64
dtype: object

In [13]:
# data
data['month_name'] = data['enddate'].dt.strftime('%B')

data['month_name'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [15]:
data['month'].nunique()

12

In [17]:
# re.sub(r'[\W_]+', '', string_value)

In [18]:
data['approve'].unique()

array([50.      , 48.      , 45.      , 51.      , 56.      , 55.47    ,
       63.      , 58.      , 52.      , 53.      , 55.      , 54.      ,
       59.      , 57.      , 49.      , 55.333333, 56.1     , 61.      ,
       46.      , 57.46    , 60.      , 57.42    , 60.9     , 62.      ,
       55.5     , 51.666667, 56.54    , 47.      , 57.86    , 53.76    ,
       58.74    , 58.5     , 53.46    , 56.17    , 57.5     , 54.35    ,
       52.21    , 54.49    , 55.18    , 59.5     , 48.3     , 54.92    ,
       55.91    , 55.87    , 61.5     , 51.57    , 53.57    , 54.95    ,
       47.9     , 52.6     , 51.5     , 52.1     , 53.43    , 53.5     ,
       53.98    , 53.83    , 43.      , 50.14    , 46.5     , 51.26    ,
       51.8     , 44.      , 46.07    , 40.      , 41.2     , 49.06    ,
       42.      , 45.8     , 45.94    , 39.      , 43.5     , 47.03    ,
       44.36    , 44.21    , 41.      , 48.5     , 45.77    , 38.      ,
       45.5     , 39.6     , 48.35    , 46.08    , 

In [19]:
data.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp,month,month_name
0,Joe Biden,All polls,3/23/2022,1/19/2021,2021-01-21,Morning Consult,B,15000.0,a,0.259361,...,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74272,139491,1/28/2021,16:23:18 23 Mar 2022,1,January
1,Joe Biden,All polls,3/23/2022,1/19/2021,2021-01-21,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.338188,...,39.216619,NaN,T,https://www.rasmussenreports.com/public_conten...,74247,139395,1/22/2021,16:23:18 23 Mar 2022,1,January
2,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-21,YouGov,B+,1516.0,a,1.245401,...,28.659628,NaN,NaN,https://docs.cdn.yougov.com/u3h9dresbn/2021012...,74327,139570,2/2/2021,16:23:18 23 Mar 2022,1,January
3,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-22,Morning Consult,B,15000.0,a,0.233261,...,31.211739,NaN,T,https://morningconsult.com/form/global-leader-...,74273,139492,1/28/2021,16:23:18 23 Mar 2022,1,January
4,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-21,Morning Consult,B,1993.0,rv,0.092978,...,34.211739,NaN,NaN,https://assets.morningconsult.com/wp-uploads/2...,74246,139394,1/22/2021,16:23:18 23 Mar 2022,1,January


In [20]:
data.groupby(['month', 'month_name'])['approve'].std()

month  month_name
1      January       3.951662
2      February      3.500613
3      March         3.643757
4      April         3.407253
5      May           2.970734
6      June          2.809410
7      July          2.976808
8      August        3.090370
9      September     2.525531
10     October       2.819754
11     November      2.391657
12     December      2.479371
Name: approve, dtype: float64

In [21]:
std_mean = data.groupby(['month', 'month_name'])['approve'].describe()
std_mean

,,count,mean,std,min,25%,50%,75%,max
month,month_name,,,,,,,,
1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0
2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0
3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0
4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0
5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0
6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0
7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0
8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0
9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0


In [22]:
std_mean['SEM'] = std_mean.apply( lambda x: x['std'] / np.sqrt(x['count']), axis=1 )
std_mean


,,count,mean,std,min,25%,50%,75%,max,SEM
month,month_name,,,,,,,,,
1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0,0.444597
2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0,0.250684
3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0,0.242917
4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0,0.225158
5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0,0.202602
6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0,0.194331
7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0,0.214833
8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0,0.197841
9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0,0.156929


In [23]:
alpha_90 = 1.645
alpha_95 = 1.960

std_mean['ci_90'] = std_mean.apply( lambda x: alpha_90 *  x['SEM'], axis=1 )
std_mean['ci_95'] = std_mean.apply( lambda x: alpha_95 *  x['SEM'], axis=1 )

std_mean

,,count,mean,std,min,25%,50%,75%,max,SEM,ci_90,ci_95
month,month_name,,,,,,,,,,,
1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0,0.444597,0.731362,0.871410
2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0,0.250684,0.412375,0.491341
3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0,0.242917,0.399599,0.476118
4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0,0.225158,0.370384,0.441309
5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0,0.202602,0.333281,0.397101
6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0,0.194331,0.319674,0.380889
7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0,0.214833,0.353400,0.421072
8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0,0.197841,0.325448,0.387768
9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0,0.156929,0.258148,0.307581


In [24]:
std_mean['std_1_pos'] = std_mean.apply( lambda x: (x['mean'] + x['std']), axis=1 )
std_mean['std_1_neg'] = std_mean.apply( lambda x: (x['mean'] - x['std']), axis=1 )

std_mean['std_2_pos'] = std_mean.apply( lambda x: (x['mean'] + (2*x['std'])), axis=1 )
std_mean['std_2_neg'] = std_mean.apply( lambda x: (x['mean'] - (2*x['std'])), axis=1 )

std_mean['std_3_pos'] = std_mean.apply( lambda x: (x['mean'] + (3*x['std'])), axis=1 )
std_mean['std_3_neg'] = std_mean.apply( lambda x: (x['mean'] - (3*x['std'])), axis=1 )

std_mean

,,count,mean,std,min,25%,50%,75%,max,SEM,ci_90,ci_95,std_1_pos,std_1_neg,std_2_pos,std_2_neg,std_3_pos,std_3_neg
month,month_name,,,,,,,,,,,,,,,,,
1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0,0.444597,0.731362,0.871410,57.506177,49.602853,61.457838,45.651191,65.409500,41.699529
2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0,0.250684,0.412375,0.491341,57.449194,50.447968,60.949807,46.947355,64.450420,43.446742
3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0,0.242917,0.399599,0.476118,57.735490,50.447977,61.379246,46.804220,65.023003,43.160464
4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0,0.225158,0.370384,0.441309,57.087603,50.273096,60.494856,46.865843,63.902109,43.458589
5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0,0.202602,0.333281,0.397101,56.794641,50.853173,59.765375,47.882439,62.736109,44.911705
6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0,0.194331,0.319674,0.380889,55.264817,49.645997,58.074226,46.836587,60.883636,44.027177
7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0,0.214833,0.353400,0.421072,54.465767,48.512150,57.442575,45.535342,60.419383,42.558534
8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0,0.197841,0.325448,0.387768,52.303731,46.122991,55.394100,43.032621,58.484470,39.942251
9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0,0.156929,0.258148,0.307581,48.792789,43.741728,51.318320,41.216198,53.843850,38.690667


In [25]:
std_mean.reset_index(inplace=True)

In [26]:
std_mean

,month,month_name,count,mean,std,min,25%,50%,75%,max,SEM,ci_90,ci_95,std_1_pos,std_1_neg,std_2_pos,std_2_neg,std_3_pos,std_3_neg
0,1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0,0.444597,0.731362,0.871410,57.506177,49.602853,61.457838,45.651191,65.409500,41.699529
1,2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0,0.250684,0.412375,0.491341,57.449194,50.447968,60.949807,46.947355,64.450420,43.446742
2,3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0,0.242917,0.399599,0.476118,57.735490,50.447977,61.379246,46.804220,65.023003,43.160464
3,4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0,0.225158,0.370384,0.441309,57.087603,50.273096,60.494856,46.865843,63.902109,43.458589
4,5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0,0.202602,0.333281,0.397101,56.794641,50.853173,59.765375,47.882439,62.736109,44.911705
5,6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0,0.194331,0.319674,0.380889,55.264817,49.645997,58.074226,46.836587,60.883636,44.027177
6,7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0,0.214833,0.353400,0.421072,54.465767,48.512150,57.442575,45.535342,60.419383,42.558534
7,8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0,0.197841,0.325448,0.387768,52.303731,46.122991,55.394100,43.032621,58.484470,39.942251
8,9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0,0.156929,0.258148,0.307581,48.792789,43.741728,51.318320,41.216198,53.843850,38.690667
9,10,October,245.0,44.432327,2.819754,37.0,43.0,45.0,46.000,53.0,0.180147,0.296343,0.353089,47.252080,41.612573,50.071834,38.792819,52.891588,35.973065


In [27]:
std_mean.columns

Index(['month', 'month_name', 'count', 'mean', 'std', 'min', '25%', '50%',
       '75%', 'max', 'SEM', 'ci_90', 'ci_95', 'std_1_pos', 'std_1_neg',
       'std_2_pos', 'std_2_neg', 'std_3_pos', 'std_3_neg'],
      dtype='object')

In [29]:
# positive counts within standard deviation    
data['count_std_1_pos'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] > std_mean[std_mean['month_name'] == x['month_name']]['mean']) 
                                         & 
                                         (x['approve'] <= std_mean[std_mean['month_name'] == x['month_name']]['std_1_pos'])
                                         ).all()
                                     else 0
                                     , axis=1)


data['count_std_2_pos'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] > std_mean[std_mean['month_name'] == x['month_name']]['std_1_pos']) 
                                         & 
                                         (x['approve'] <= std_mean[std_mean['month_name'] == x['month_name']]['std_2_pos'])
                                         ).all()
                                     else 0
                                     , axis=1)

data['count_std_3_pos'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] > std_mean[std_mean['month_name'] == x['month_name']]['std_2_pos']) 
                                         & 
                                         (x['approve'] <= std_mean[std_mean['month_name'] == x['month_name']]['std_3_pos'])
                                         ).all()
                                     else 0
                                     , axis=1)


# negative counts within standard deviation    
data['count_std_1_neg'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] < std_mean[std_mean['month_name'] == x['month_name']]['mean']) 
                                         & 
                                         (x['approve'] >= std_mean[std_mean['month_name'] == x['month_name']]['std_1_neg'])
                                         ).all()
                                     else 0
                                     , axis=1)



data['count_std_2_neg'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] < std_mean[std_mean['month_name'] == x['month_name']]['std_1_neg']) 
                                         & 
                                         (x['approve'] >= std_mean[std_mean['month_name'] == x['month_name']]['std_2_neg'])
                                         ).all()
                                     else 0
                                     , axis=1)


data['count_std_3_neg'] = data.apply(lambda x: 
                                     1 
                                     if (
                                         (x['approve'] < std_mean[std_mean['month_name'] == x['month_name']]['std_2_neg']) 
                                         & 
                                         (x['approve'] >= std_mean[std_mean['month_name'] == x['month_name']]['std_3_neg'])
                                         ).all()
                                     else 0
                                     , axis=1)




data


,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,createddate,timestamp,month,month_name,count_std_1_pos,count_std_2_pos,count_std_3_pos,count_std_1_neg,count_std_2_neg,count_std_3_neg
0,Joe Biden,All polls,3/23/2022,1/19/2021,2021-01-21,Morning Consult,B,15000.0,a,0.259361,...,1/28/2021,16:23:18 23 Mar 2022,1,January,0,0,0,1,0,0
1,Joe Biden,All polls,3/23/2022,1/19/2021,2021-01-21,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.338188,...,1/22/2021,16:23:18 23 Mar 2022,1,January,0,0,0,0,1,0
2,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-21,YouGov,B+,1516.0,a,1.245401,...,2/2/2021,16:23:18 23 Mar 2022,1,January,0,0,0,0,0,1
3,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-22,Morning Consult,B,15000.0,a,0.233261,...,1/28/2021,16:23:18 23 Mar 2022,1,January,0,0,0,1,0,0
4,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-21,Morning Consult,B,1993.0,rv,0.092978,...,1/22/2021,16:23:18 23 Mar 2022,1,January,1,0,0,0,0,0
5,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-21,Ipsos,B-,1115.0,a,1.101398,...,1/22/2021,16:23:18 23 Mar 2022,1,January,1,0,0,0,0,0
6,Joe Biden,All polls,3/23/2022,1/21/2021,2021-01-22,HarrisX,B+,941.0,rv,1.041367,...,1/25/2021,16:23:18 23 Mar 2022,1,January,0,0,1,0,0,0
7,Joe Biden,All polls,3/23/2022,1/21/2021,2021-01-23,RMG Research,B-,1200.0,rv,0.880674,...,1/28/2021,16:23:18 23 Mar 2022,1,January,0,1,0,0,0,0
8,Joe Biden,All polls,3/23/2022,1/21/2021,2021-01-23,Morning Consult,B,15000.0,a,0.214604,...,1/28/2021,16:23:18 23 Mar 2022,1,January,0,0,0,1,0,0
9,Joe Biden,All polls,3/23/2022,1/20/2021,2021-01-24,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.177644,...,1/25/2021,16:23:18 23 Mar 2022,1,January,0,0,0,0,1,0


In [30]:
# subset just the counts for std groups
monthly_group = data.groupby(['month_name']).sum()

monthly = monthly_group[['count_std_1_pos', 'count_std_2_pos', 'count_std_3_pos', 'count_std_1_neg', 'count_std_2_neg', 'count_std_3_neg']]

monthly


,count_std_1_pos,count_std_2_pos,count_std_3_pos,count_std_1_neg,count_std_2_neg,count_std_3_neg
month_name,,,,,,
April,106,26,6,42,49,0
August,87,30,2,76,40,9
December,85,19,8,50,47,2
February,89,25,8,31,40,2
January,30,11,2,20,14,2
July,99,5,12,48,26,2
June,85,16,4,68,32,0
March,68,35,4,70,48,0
May,114,13,8,46,32,0


In [31]:
data.columns

Index(['president', 'subgroup', 'modeldate', 'startdate', 'enddate',
       'pollster', 'grade', 'samplesize', 'population', 'weight', 'influence',
       'approve', 'disapprove', 'adjusted_approve', 'adjusted_disapprove',
       'multiversions', 'tracking', 'url', 'poll_id', 'question_id',
       'createddate', 'timestamp', 'month', 'month_name', 'count_std_1_pos',
       'count_std_2_pos', 'count_std_3_pos', 'count_std_1_neg',
       'count_std_2_neg', 'count_std_3_neg'],
      dtype='object')

In [32]:
data_subset = data[['month_name', 'month', 'grade', 'enddate', 'pollster', 'samplesize', 'approve', 'influence', 'disapprove']]
data_subset

,month_name,month,grade,enddate,pollster,samplesize,approve,influence,disapprove
0,January,1,B,2021-01-21,Morning Consult,15000.0,50.00,0.0,28.00
1,January,1,B,2021-01-21,Rasmussen Reports/Pulse Opinion Research,1500.0,48.00,0.0,45.00
2,January,1,B+,2021-01-21,YouGov,1516.0,45.00,0.0,28.00
3,January,1,B,2021-01-22,Morning Consult,15000.0,51.00,0.0,28.00
4,January,1,B,2021-01-21,Morning Consult,1993.0,56.00,0.0,31.00
5,January,1,B-,2021-01-21,Ipsos,1115.0,55.47,0.0,31.59
6,January,1,B+,2021-01-22,HarrisX,941.0,63.00,0.0,37.00
7,January,1,B-,2021-01-23,RMG Research,1200.0,58.00,0.0,32.00
8,January,1,B,2021-01-23,Morning Consult,15000.0,52.00,0.0,29.00
9,January,1,B,2021-01-24,Rasmussen Reports/Pulse Opinion Research,1500.0,48.00,0.0,47.00


In [33]:
std_mean = pd.merge(std_mean, monthly, on='month_name', how='left')
std_mean

,month,month_name,count,mean,std,min,25%,50%,75%,max,...,std_2_pos,std_2_neg,std_3_pos,std_3_neg,count_std_1_pos,count_std_2_pos,count_std_3_pos,count_std_1_neg,count_std_2_neg,count_std_3_neg
0,1,January,79.0,53.554515,3.951662,45.0,50.0,54.0,56.000,63.0,...,61.457838,45.651191,65.409500,41.699529,30,11,2,20,14,2
1,2,February,195.0,53.948581,3.500613,46.0,51.0,54.0,56.000,62.0,...,60.949807,46.947355,64.450420,43.446742,89,25,8,31,40,2
2,3,March,225.0,54.091733,3.643757,47.0,51.0,54.0,56.000,62.0,...,61.379246,46.804220,65.023003,43.160464,68,35,4,70,48,0
3,4,April,229.0,53.680349,3.407253,47.0,51.0,54.0,55.000,61.0,...,60.494856,46.865843,63.902109,43.458589,106,26,6,42,49,0
4,5,May,215.0,53.823907,2.970734,48.0,52.0,54.0,55.955,63.0,...,59.765375,47.882439,62.736109,44.911705,114,13,8,46,32,0
5,6,June,209.0,52.455407,2.809410,47.9,50.0,53.0,54.000,61.0,...,58.074226,46.836587,60.883636,44.027177,85,16,4,68,32,0
6,7,July,192.0,51.488958,2.976808,43.0,49.0,52.0,53.000,60.0,...,57.442575,45.535342,60.419383,42.558534,99,5,12,48,26,2
7,8,August,244.0,49.213361,3.090370,40.0,47.0,49.0,51.000,56.0,...,55.394100,43.032621,58.484470,39.942251,87,30,2,76,40,9
8,9,September,259.0,46.267259,2.525531,39.0,45.0,46.0,47.515,55.0,...,51.318320,41.216198,53.843850,38.690667,90,30,5,98,26,8
9,10,October,245.0,44.432327,2.819754,37.0,43.0,45.0,46.000,53.0,...,50.071834,38.792819,52.891588,35.973065,117,20,2,71,27,6


In [34]:
# std_mean.to_json('data/std_mean.json', orient='table')
std_mean.to_json('data/std_mean2-2021.json', orient='index')

In [35]:
std_mean.to_csv('data/std_mean-2021.csv')

In [36]:
data_subset.to_csv('data/data_subset-2021.csv')